In [1]:
import pandas as pd

In [7]:
df=pd.read_json("processed_crypto.json")
df_ripple=df[df['symbol']=='XRP']

In [10]:
df_date_created=df_ripple.sort_values('created_utc')
df_date_created['created_utc']=df_date_created['created_utc'].str.slice(0,10)
df_date_created['sma20']=df_date_created['open'].rolling(window=20).mean()
df_date_created['high-low']=df_date_created['sma20'].shift(-1)-df_date_created['sma20']

In [11]:
df_date_created['buy-sell'] = df_date_created['high-low'].apply(lambda x: 0 if x <=0 else 1)

In [12]:
train_x=df_date_created[ (df_date_created['created_utc'].str.slice(0,4)== '2018')]
train_xx=train_x
train_y=train_x['buy-sell']
train_x=train_x.drop(columns=['fin_subheading','news_title','news_url','reddit_title','symbol','tweet_text','tweet_text','asset_name','created_utc','reddit_domain','tweet_hashtags','fin_title','buy-sell',])


In [13]:
test_x=df_date_created[ (df_date_created['created_utc'].str.slice(0,4)== '2019') ]
test_xx=test_x
test_y=test_x['buy-sell']
test_x=test_x.drop(columns=['fin_subheading','news_title','news_url','reddit_title','symbol','tweet_text','tweet_text','asset_name','created_utc','reddit_domain','tweet_hashtags','fin_title','buy-sell'])


In [14]:
train_x=train_x.fillna(train_x.mean())
test_x=test_x.fillna(test_x.mean())

In [15]:
from sklearn.linear_model import SGDClassifier

In [16]:
model=SGDClassifier()

In [17]:
model.fit(train_x,train_y)

/Users/mananparasher/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:144: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=None,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

In [18]:
predict=model.predict(test_x)

In [19]:
from sklearn.metrics import classification_report

In [20]:
print(classification_report(list(test_y),predict))

              precision    recall  f1-score   support

           0       0.57      1.00      0.72       680
           1       0.00      0.00      0.00       519

   micro avg       0.57      0.57      0.57      1199
   macro avg       0.28      0.50      0.36      1199
weighted avg       0.32      0.57      0.41      1199



/Users/mananparasher/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [21]:
from sklearn.metrics import accuracy_score

In [22]:
print(accuracy_score(list(test_y),predict))

0.567139282735613
